In [1]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
from scipy import signal
from matplotlib import cm
import hashlib

In [2]:
mars = False
base_dir = "../../data/lunar/training"
if mars:
    base_dir = "./data/mars/training"

data_dir = "/data/S12_GradeA"
if mars:
    data_dir = "/data"

catalog_file = base_dir + "/catalogs/apollo12_catalog_GradeA_final.csv"
if mars:
    catalog_file = base_dir + "/catalogs/Mars_InSight_training_catalog_final.csv"

catalog = pd.read_csv(catalog_file)
catalog

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type
0,xa.s12.00.mhz.1970-01-19HR00_evid00002,1970-01-19T20:25:00.000000,73500.0,evid00002,impact_mq
1,xa.s12.00.mhz.1970-03-25HR00_evid00003,1970-03-25T03:32:00.000000,12720.0,evid00003,impact_mq
2,xa.s12.00.mhz.1970-03-26HR00_evid00004,1970-03-26T20:17:00.000000,73020.0,evid00004,impact_mq
3,xa.s12.00.mhz.1970-04-25HR00_evid00006,1970-04-25T01:14:00.000000,4440.0,evid00006,impact_mq
4,xa.s12.00.mhz.1970-04-26HR00_evid00007,1970-04-26T14:29:00.000000,52140.0,evid00007,deep_mq
...,...,...,...,...,...
71,xa.s12.00.mhz.1974-10-14HR00_evid00156,1974-10-14T17:43:00.000000,63780.0,evid00156,impact_mq
72,xa.s12.00.mhz.1975-04-12HR00_evid00191,1975-04-12T18:15:00.000000,65700.0,evid00191,impact_mq
73,xa.s12.00.mhz.1975-05-04HR00_evid00192,1975-05-04T10:05:00.000000,36300.0,evid00192,impact_mq
74,xa.s12.00.mhz.1975-06-24HR00_evid00196,1975-06-24T16:03:00.000000,57780.0,evid00196,impact_mq


In [3]:
import numpy as np

def pad_spectrogram(s, target_shape):
    current_height, current_width = s.shape
    target_height, target_width = target_shape
    
    # Calculate how much padding is needed for height and width
    pad_height = target_height - current_height
    pad_width = target_width - current_width
    
    # Apply padding if needed
    if pad_height > 0 or pad_width > 0:
        s_padded = np.pad(s, 
                          ((0, pad_height), (0, pad_width)),  # Pad along height and width
                          mode='constant', constant_values=0)
    else:
        s_padded = s  # If already the right shape
    
    return s_padded

In [4]:
def max_subarray_sum_with_index(arr, window_width):
    # Create sliding windows
    windows = np.lib.stride_tricks.sliding_window_view(arr, window_width)
    
    # Compute the sum of elements in each window
    window_sums = windows.sum(axis=1)
    
    # Find the index of the maximum sum
    max_index = np.argmax(window_sums)
    
    # Return the starting index of the maximum sum window
    return max_index

In [5]:
hashes = set()
X = []
y = []

for idx in range(len(catalog['filename'])):
    row = catalog.iloc[idx]
    filename = row['filename']
    if mars:
        filename = filename[:-4]
    
    print("----- Reading ", filename, idx, "-----")
    # arrival_time = datetime.strptime(row['time_abs(%Y-%m-%dT%H:%M:%S.%f)'],'%Y-%m-%dT%H:%M:%S.%f')
    arrival_time = row['time_rel(sec)']
    mseed_file = f'{base_dir}/{data_dir}/{filename}.mseed'
    st = read(mseed_file)
    
    digest = hashlib.md5(open(mseed_file, 'rb').read()).hexdigest()
    if digest in hashes:
        continue
    hashes.add(digest)
    
    tr = st.traces[0].copy()
    tr_times = tr.times()
    tr_data = tr.data

    print("Arrival time:", arrival_time)

    # Set the minimum frequency
    minfreq = 0.5
    maxfreq = 1.0

    # Going to create a separate trace for the filter data
    st_filt = st.copy()
    st_filt.filter('bandpass',freqmin=minfreq,freqmax=maxfreq)
    tr_filt = st_filt.traces[0].copy()
    tr_times_filt = tr_filt.times()
    tr_data_filt = tr_filt.data

    # Start time of trace (another way to get the relative arrival time using datetime)
    f, t, sxx = signal.spectrogram(tr_data_filt, tr_filt.stats.sampling_rate)
    sxx = pad_spectrogram(sxx, (129, 2555))
    X.append(sxx)
    y.append(arrival_time)
X = np.array(X)
y = np.array(y)

----- Reading  xa.s12.00.mhz.1970-01-19HR00_evid00002 0 -----
Arrival time: 73500.0
----- Reading  xa.s12.00.mhz.1970-03-25HR00_evid00003 1 -----
Arrival time: 12720.0
----- Reading  xa.s12.00.mhz.1970-03-26HR00_evid00004 2 -----
Arrival time: 73020.0
----- Reading  xa.s12.00.mhz.1970-04-25HR00_evid00006 3 -----
Arrival time: 4440.0
----- Reading  xa.s12.00.mhz.1970-04-26HR00_evid00007 4 -----
Arrival time: 52140.0
----- Reading  xa.s12.00.mhz.1970-06-15HR00_evid00008 5 -----
Arrival time: 68400.0
----- Reading  xa.s12.00.mhz.1970-06-26HR00_evid00009 6 -----
Arrival time: 72060.0
----- Reading  xa.s12.00.mhz.1970-07-20HR00_evid00010 7 -----
Arrival time: 18360.0
----- Reading  xa.s12.00.mhz.1970-07-20HR00_evid00011 8 -----
----- Reading  xa.s12.00.mhz.1970-09-26HR00_evid00013 9 -----
Arrival time: 71820.0
----- Reading  xa.s12.00.mhz.1970-10-24HR00_evid00014 10 -----
Arrival time: 41460.0
----- Reading  xa.s12.00.mhz.1970-11-12HR00_evid00015 11 -----
Arrival time: 46200.0
----- Reading

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Assume X is your spectrogram data (samples, height, width)
# And y is your target (earthquake start time in seconds)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Load or split your data

# Ensure the input spectrograms are 3D, as CNNs expect
# Shape: (num_samples, height, width, channels) - Use 1 channel for grayscale
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

# Define CNN model
model = models.Sequential()

# Input layer (Conv2D)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(layers.MaxPooling2D((2, 2)))

# Add more Conv layers
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Flatten and add Dense layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))

# Output layer (regression task - predicting start time)
model.add(layers.Dense(1, activation='linear'))

# Compile model for regression (use mean squared error loss)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=4, validation_split=0.2)

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

c:\Users\cgera\Desktop\Coding\space_apps_2024_seismic_detection\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
 8/11 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 2304326656.0000 - mae: 41970.4844

KeyboardInterrupt: 